In [0]:
customer_dim_data = [

(1,'manish','arwal','india','N','2022-09-15','2022-09-25'),
(2,'vikash','patna','india','Y','2023-08-12',None),
(3,'nikita','delhi','india','Y','2023-09-10',None),
(4,'rakesh','jaipur','india','Y','2023-06-10',None),
(5,'ayush','NY','USA','Y','2023-06-10',None),
(1,'manish','gurgaon','india','Y','2022-09-25',None),
]

customer_schema= ['id','name','city','country','active','effective_start_date','effective_end_date']

customer_dim_df = spark.createDataFrame(data= customer_dim_data,schema=customer_schema)

sales_data = [

(1,1,'manish','2023-01-16','gurgaon','india',380),
(77,1,'manish','2023-03-11','bangalore','india',300),
(12,3,'nikita','2023-09-20','delhi','india',127),
(54,4,'rakesh','2023-08-10','jaipur','india',321),
(65,5,'ayush','2023-09-07','mosco','russia',765),
(89,6,'rajat','2023-08-10','jaipur','india',321)
]

sales_schema = ['sales_id', 'customer_id','customer_name', 'sales_date', 'food_delivery_address','food_delivery_country', 'food_cost']

sales_df = spark.createDataFrame(data=sales_data,schema=sales_schema)

In [0]:
sales_df.show()

In [0]:
customer_dim_df.show()

In [0]:
joined_df = customer_dim_df.join(sales_df,customer_dim_df.name == sales_df.customer_name,'left')

In [0]:
display(joined_df)

In [0]:
from pyspark.sql.functions import col,lit

In [0]:
new_records_df = joined_df.filter((col('food_delivery_address') != col('city')) & (col('active') == 'Y'))\
                .withColumn('active',lit('Y'))\
                .withColumn('effective_start_date',col('sales_date'))\
                .withColumn('effective_end_date',lit(None))\
                .select(
                    'customer_id',
                    'customer_name',
                    col('food_delivery_address').alias('city'),
                    'food_delivery_country',
                    'active',
                    'effective_start_date',
                    'effective_end_date'
                )

display(new_records_df)

update old records

In [0]:
old_records = joined_df.filter(
    (col('food_delivery_address') != col('city')) & (col('active') == 'Y'))\
    .withColumn('active',lit('N'))\
    .withColumn('effective_end_date',col('sales_date'))\
    .select(
        'customer_id',
        'customer_name',
        'city',
        'food_delivery_country',
        'active',
        'effective_start_date',
        'effective_end_date'
    )

display(old_records)

find new records and insert them

In [0]:
new_customers_df = sales_df.join(customer_dim_df,sales_df['customer_id']==customer_dim_df['id'],'leftanti')\
              .withColumn('active',lit('Y'))\
              .withColumn('effective_start_date',col('sales_date'))\
              .withColumn('effective_end_date',lit(None))\
              .select(
                'customer_id',
                'customer_name',
                'food_delivery_address',
                'food_delivery_country',
                'active',
                'effective_start_date',
                'effective_end_date'
              )

display(new_customers_df)

Merge dataframes

In [0]:
final_records = customer_dim_df.union(new_records_df).union(old_records).union(new_customers_df)

display(final_records)

Remove Duplicate date

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

In [0]:
window = Window.partitionBy('id','active').orderBy(col('effective_start_date').desc())

final_records.withColumn('rnk',rank().over(window)).filter(~((col('active')=='Y') & (col('rnk')>=2))).drop('rnk').show()